In [2]:
# !pip install kaggle
# !cd ~/.kaggle
# !mv ~/Downloads/kaggle.json ./
# !chmod 600 ./kaggle.json


In [9]:
!kaggle competitions download -c caltech-cs155-2022-mp1

 94%|███████████████████████████████████▊  | 12.0M/12.7M [00:00<00:00, 19.0MB/s]
100%|██████████████████████████████████████| 12.7M/12.7M [00:00<00:00, 18.3MB/s]


In [11]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
